In [0]:
pip install pyspark

Python interpreter will be restarted.
Python interpreter will be restarted.


### About the Dataset



This dataset provides comprehensive information about Spotify tracks encompassing a diverse collection of 125 genres. It has been compiled and cleaned using Spotify's Web API and Python. Presented in CSV format, this dataset is easily accessible and amenable to analysis. The dataset comprises multiple columns, each representing distinctive audio features associated with individual tracks.



#### Knowing the Data

| Column Name      | Description                                                                                                                     |
|------------------|---------------------------------------------------------------------------------------------------------------------------------|
| artists          | The name(s) of the artist(s) associated with the track. (String)                                                                |
| album_name       | The name of the album that the track belongs to. (String)                                                                       |
| track_name       | The name of the track. (String)                                                                                                 |
| popularity       | The popularity score of the track on Spotify, ranging from 0 to 100. (Integer)                                                  |
| duration_ms      | The duration of the track in milliseconds. (Integer)                                                                            |
| explicit         | A boolean value indicating whether the track contains explicit content. (Boolean)                                               |
| danceability     | A score ranging from 0 to 1 that represents how suitable a track is for dancing based on various musical elements. (Float)      |
| energy           | A measure of the intensity and activity of a track, ranging from 0 to 1. (Float)                                                |
| key              | The key of the track represented by an integer value. (Integer)                                                                 |
| loudness         | The loudness of the track in decibels (dB). (Float)                                                                             |
| mode             | The tonal mode of the track, represented by an integer value (0 for minor, 1 for major). (Integer)                              |
| speechiness      | A score ranging from 0 to 1 that represents the presence of spoken words in a track. (Float)                                    |
| acousticness     | A score ranging from 0 to 1 that represents the extent to which a track possesses an acoustic quality. (Float)                  |
| instrumentalness | A score ranging from 0 to 1 that represents the likelihood of a track being instrumental. (Float)                               |
| liveness         | A score ranging from 0 to 1 that represents the presence of an audience during the recording or performance of a track. (Float) |
| valence          | A score ranging from 0 to 1 that represents the musical positiveness conveyed by a track. (Float)                               |
| tempo            | The tempo of the track in beats per minute (BPM). (Float)                                                                       |
| time_signature   | The number of beats within each bar of the track. (Integer)                                                                     |
| track_genre      | The genre of the track. (String)                                                                                                |

In [0]:
# Load necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc



# Create a Spark session
spark = SparkSession.builder.appName("SpotifyAnalysis").getOrCreate()

# Read the CSV file into a DataFrame
spotify_df = spark.read.csv('/FileStore/tables/train-1.csv',header=True,inferSchema=True)

# Load CSV data into a DataFrame
#spotify_df = spark.read.csv("D:/M.Tech BITS Pilani/Semester 3/Big Data Systems (S1-23_DSEOGZG522)/Assignment/Spotify/train.csv", header=True, inferSchema=True)



Data at Glance

In [0]:
spotify_df.show(5)

+----------+--------------------+--------------------+--------------------+--------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+
|Unnamed: 0|            track_id|             artists|          album_name|          track_name|popularity|duration_ms|explicit|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|track_genre|
+----------+--------------------+--------------------+--------------------+--------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+
|         0|5SuOikwiRyPMVoIQD...|         Gen Hoshino|              Comedy|              Comedy|        73|     230666|   False|       0.676| 0.461|  1|  -6.746|   0|      0.143|      0.0322|         1.01E-6|   0.358|

####Query 1: Identify High-Potential Genres for Marketing and Promotion

In [0]:
# Cache the DataFrame for better performance
spotify_df.cache()

# Find the top genres based on the average popularity of tracks within each genre
high_potential_genres_query = (
    spotify_df
    .groupBy("track_genre")
    .agg({"popularity": "avg"})
    .orderBy(desc("avg(popularity)"))
    .limit(10)
)

high_potential_genres_query.show()


+-----------+------------------+
|track_genre|   avg(popularity)|
+-----------+------------------+
|   pop-film|59.287575150300604|
|      k-pop|            56.896|
|      chill|            53.651|
|        sad|            52.379|
|     grunge|            49.594|
|     indian|            49.539|
|      anime|            48.772|
|        emo|            48.128|
|  sertanejo|            47.866|
|        pop|            47.576|
+-----------+------------------+



#####Business value: Got the insight about the tracks which are more popular and listen by the users. These tracks are best suited for the promotional advertisements. 

####Query2: Identify Top Artists and Genres by Average Popularity

This query helps you identify the top artists and genres with the highest average popularity.

In [0]:


top_artists_genres_query = (
    spotify_df
    .groupBy("artists", "track_genre")
    .agg({"popularity": "avg"})
    .orderBy(desc("avg(popularity)"))
    .limit(10)
)

top_artists_genres_query.show()


+--------------------+-----------+---------------+
|             artists|track_genre|avg(popularity)|
+--------------------+-----------+---------------+
|Sam Smith;Kim Petras|      dance|          100.0|
|Sam Smith;Kim Petras|        pop|          100.0|
|    Bizarrap;Quevedo|    hip-hop|           99.0|
|       Manuel Turizo|      latin|           98.0|
|       Manuel Turizo|     latino|           98.0|
|       Manuel Turizo|     reggae|           98.0|
|       Manuel Turizo|  reggaeton|           98.0|
|Bad Bunny;Chencho...|      latin|           97.0|
|Bad Bunny;Chencho...|     reggae|           97.0|
|Bad Bunny;Chencho...|     latino|           97.0|
+--------------------+-----------+---------------+



#####Business value: It gives us the insight, which artist and tracks are more popular among the users. This insight may be helpful for the business promoter like artist to be approached. Focusing on these artists and genres in marketing and promotions could lead to increased user engagement.

Query 2: Analyze the Impact of Danceability and Energy on Track Popularity

####Query3: Identify the Most Common Keys and Modes

Knowing the most common keys and modes in your dataset can help in creating playlists.

In [0]:
common_keys_modes_query = (
    spotify_df
    .groupBy("key", "mode")
    .count()
    .orderBy(desc("count"))
)

common_keys_modes_query.show(10)


+---+----+-----+
|key|mode|count|
+---+----+-----+
|  0|   1|10179|
|  7|   1|10130|
|  2|   1| 9052|
|  1|   1| 7164|
|  9|   1| 6857|
|  8|   1| 5436|
|  5|   1| 5336|
| 11|   0| 5145|
|  4|   0| 4724|
|  9|   0| 4453|
+---+----+-----+
only showing top 10 rows



#####Business value: It helps us to get insight which key and mode align with popular music trends, contributing to user engagement and satisfaction.

####Query4: Track name and Popularity on the basis of specific Artist

In [0]:
from pyspark.sql.functions import col

# OLTP Query: Find tracks by a specific artist with popularity greater than a threshold
olap_query_result = (
    spotify_df
    .filter((col("artists") == "Harry Styles") & (col("popularity") > 80))
    .select("track_name", "popularity")
)

olap_query_result.show()


+----------------+----------+
|      track_name|popularity|
+----------------+----------+
|       As It Was|        95|
|Watermelon Sugar|        89|
|       As It Was|        92|
+----------------+----------+



####Query5: Top Genres by Popularity

In [0]:


top_genres_df = spotify_df.groupBy("track_genre").count().orderBy(col("count").desc()).limit(10)
top_genres_df.show()


+-----------+-----+
|track_genre|count|
+-----------+-----+
|      blues| 1000|
|  dancehall| 1000|
|  breakbeat| 1000|
|     brazil| 1000|
|alternative| 1000|
|    country| 1000|
| deep-house| 1000|
|    ambient| 1000|
|      chill| 1000|
|      anime| 1000|
+-----------+-----+

